In [1]:
# code by Peter Solis
# dependencies
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# do all the formatting, this is copied from the supervised learning models
# read in data
df = pd.read_csv('../01 - Data Crunching/dataset.csv')

# translating our target into categorical values for these models to work with
# https://www.codementor.io/@agarrahul01/multiclass-classification-using-random-forest-on-scikit-learn-library-hkk4lwawu was referenced for how to make a tree / forest model with these in mind
target_names = pd.factorize(df['Target'])[1]
df['Target'] = pd.factorize(df['Target'])[0]
for i in range(len(target_names)):
    print(f'{i} - {target_names[i]}')
    
# split based on data type
categorical_df = df.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18]].copy()
numerical_df = df.iloc[:,[2,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]].copy()
y = df[['Target']].copy()

# make non-binary categorical columns clearly categorical - you may want to also bin these in some cases
# more info on what each category means can be found here:
# https://www.mdpi.com/2306-5729/7/11/146
def categorize(cell):
    return f'cat_{cell}'
col_to_adjust = ['Marital status',
                 'Application mode',
                 'Course',
                 'Previous qualification',
                 'Nacionality',
                 "Mother's qualification",
                 "Father's qualification",
                 "Mother's occupation",
                 "Father's occupation"]
for col in col_to_adjust:
    categorical_df[col] = categorical_df[col].apply(categorize)

# finish pre-processing categorical data
# I referenced the tables of most important variables found in the cell above, and decided to exclude some categorical data that didn't play a huge role and would greatly increase the number of columns
# important categorical data with tons of values
to_drop = ["Application mode",
           "Nacionality",
           "Mother's qualification",
           "Father's qualification"]
cat_df_2 = pd.get_dummies(categorical_df.drop(columns = to_drop, axis = 1))
# try using PCA to deal with the huge number of values, we'll give it twice the number of columns of the actual values
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
pca_cat_df = pd.DataFrame(pca.fit_transform(cat_df_2), columns = ['PCA1','PCA2','PCA3','PCA4','PCA5','PCA6','PCA7','PCA8','PCA9','PCA10'])

# these are the categorical columns that stay as one column
others = ['Daytime/evening attendance',
          'Displaced',
          'Educational special needs',
          'Debtor',
          'Tuition fees up to date',
          'Gender',
          'Scholarship holder',
          'International']
parsed_cat_df = pd.concat([pca_cat_df, categorical_df[others]], axis = 1)

# finish preprocessing by scaling and doing PCA
scaled_num_df = pd.DataFrame(StandardScaler().fit_transform(numerical_df), columns = numerical_df.columns)
final_df = pd.concat([parsed_cat_df, scaled_num_df], axis = 1)
final_df.head()

0 - Dropout
1 - Graduate
2 - Enrolled


,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,-0.162155,0.028002,-0.871987,-0.003519,-0.282296,-0.504824,-0.168469,-0.306896,0.555613,-0.052945,...,-0.199273,-0.282442,-2.838337,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761
1,-0.230439,-0.508801,-0.888205,-0.126158,-0.399622,0.015986,-0.086921,-0.037612,-0.101045,0.013282,...,-0.199273,-0.282442,-0.105726,-0.522682,0.518904,0.659562,-0.199441,0.876222,-1.105222,0.347199
2,0.442844,0.679992,-1.137401,-0.198085,-0.015669,0.027037,-0.076970,-0.163251,0.026286,0.334927,...,-0.199273,-0.282442,-0.105726,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761
3,-0.720106,-0.190385,-0.002961,-0.299306,-0.633617,-0.209809,-0.348117,-0.191423,0.111372,-0.142614,...,-0.199273,-0.282442,-0.105726,0.490616,0.187165,0.416450,-0.199441,-0.813253,-1.466871,-1.375511
4,1.459906,0.612459,0.796140,-0.487460,-0.180109,-0.687948,0.330021,-0.474441,-0.070128,0.051929,...,-0.199273,-0.282442,-0.105726,-0.522682,0.518904,0.531608,-0.199441,0.876222,-1.105222,0.347199


In [7]:
df

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,0
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,1
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,0
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,1
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,15,1,1,1,1,1,6,...,0,6,8,5,12.666667,0,15.5,2.8,-4.06,1
4420,1,1,2,15,1,1,19,1,1,10,...,0,6,6,2,11.000000,0,11.1,0.6,2.02,0
4421,1,1,1,12,1,1,1,22,27,10,...,0,8,9,1,13.500000,0,13.9,-0.3,0.79,0
4422,1,1,1,9,1,1,1,22,27,8,...,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,1


In [13]:
# for the neural network, we do need to split y into 3 columns
final_y = pd.get_dummies(y['Target'])
final_y.head()

,0,1,2
0,1,0,0
1,0,1,0
2,1,0,0
3,0,1,0
4,0,1,0


In [14]:
# split data
X_train, X_test, y_train, y_test = train_test_split(final_df, final_y, random_state = 0)

In [26]:
# model 1 - Sequential, 70 > 30 neurons, 100 epochs
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1] # number of columns
hidden_nodes_layer1 = 70
hidden_nodes_layer2 = 30
nn = Sequential()

# layers
nn.add(Dense(units = hidden_nodes_layer1, activation = 'relu', input_dim = number_input_features))
nn.add(Dense(units = hidden_nodes_layer2, activation = 'relu'))
nn.add(Dense(units = y_train.shape[1], activation = 'sigmoid'))
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 70)                2520      
                                                                 
 dense_13 (Dense)            (None, 30)                2130      
                                                                 
 dense_14 (Dense)            (None, 3)                 93        
                                                                 
Total params: 4743 (18.53 KB)
Trainable params: 4743 (18.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# compile & train
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
nn.fit(X_train, y_train, epochs = 100)

Epoch 1/100
104/104 [==============================] - 1s 1ms/step - loss: 0.5300 - accuracy: 0.6519
Epoch 2/100
104/104 [==============================] - 0s 1ms/step - loss: 0.3913 - accuracy: 0.7266
Epoch 3/100
104/104 [==============================] - 0s 2ms/step - loss: 0.3662 - accuracy: 0.7483
Epoch 4/100
104/104 [==============================] - 0s 1ms/step - loss: 0.3547 - accuracy: 0.7634
Epoch 5/100
104/104 [==============================] - 0s 1ms/step - loss: 0.3479 - accuracy: 0.7719
Epoch 6/100
104/104 [==============================] - 0s 1ms/step - loss: 0.3424 - accuracy: 0.7728
Epoch 7/100
104/104 [==============================] - 0s 1ms/step - loss: 0.3375 - accuracy: 0.7821
Epoch 8/100
104/104 [==============================] - 0s 2ms/step - loss: 0.3324 - accuracy: 0.7848
Epoch 9/100
104/104 [==============================] - 0s 2ms/step - loss: 0.3290 - accuracy: 0.7866
Epoch 10/100
104/104 [==============================] - 0s 1ms/step - loss: 0.3253 - accura

In [51]:
# predict and score
predicts = pd.DataFrame(nn.predict(X_test), columns = [0, 1, 2])

model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(classification_report(y_test.idxmax(axis = 1), predicts.idxmax(axis = 1), target_names = target_names))
confusion_matrix(y_test.idxmax(axis = 1), predicts.idxmax(axis = 1))

35/35 [==============================] - 0s 939us/step
35/35 - 0s - loss: 0.5438 - accuracy: 0.7505 - 49ms/epoch - 1ms/step
Loss: 0.5437610745429993, Accuracy: 0.7504521012306213
              precision    recall  f1-score   support

     Dropout       0.75      0.77      0.76       353
    Graduate       0.85      0.83      0.84       560
    Enrolled       0.47      0.47      0.47       193

    accuracy                           0.75      1106
   macro avg       0.69      0.69      0.69      1106
weighted avg       0.75      0.75      0.75      1106



array([[273,  33,  47],
       [ 37, 467,  56],
       [ 56,  47,  90]], dtype=int64)

In [58]:
# model 2 - Sequential, 100 > 30 neurons, 100 epochs
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1] # number of columns
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 30
nn = Sequential()

# layers
nn.add(Dense(units = hidden_nodes_layer1, activation = 'relu', input_dim = number_input_features))
nn.add(Dense(units = hidden_nodes_layer2, activation = 'relu'))
nn.add(Dense(units = y_train.shape[1], activation = 'sigmoid'))
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               3600      
                                                                 
 dense_22 (Dense)            (None, 30)                3030      
                                                                 
 dense_23 (Dense)            (None, 3)                 93        
                                                                 
Total params: 6723 (26.26 KB)
Trainable params: 6723 (26.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
# compile & train
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
nn.fit(X_train, y_train, epochs = 75)

Epoch 1/75
104/104 [==============================] - 1s 2ms/step - loss: 0.4950 - accuracy: 0.6495
Epoch 2/75
104/104 [==============================] - 0s 1ms/step - loss: 0.3844 - accuracy: 0.7450
Epoch 3/75
104/104 [==============================] - 0s 2ms/step - loss: 0.3607 - accuracy: 0.7628
Epoch 4/75
104/104 [==============================] - 0s 2ms/step - loss: 0.3486 - accuracy: 0.7703
Epoch 5/75
104/104 [==============================] - 0s 1ms/step - loss: 0.3411 - accuracy: 0.7782
Epoch 6/75
104/104 [==============================] - 0s 1ms/step - loss: 0.3350 - accuracy: 0.7752
Epoch 7/75
104/104 [==============================] - 0s 1ms/step - loss: 0.3306 - accuracy: 0.7824
Epoch 8/75
104/104 [==============================] - 0s 2ms/step - loss: 0.3238 - accuracy: 0.7905
Epoch 9/75
104/104 [==============================] - 0s 2ms/step - loss: 0.3213 - accuracy: 0.7896
Epoch 10/75
104/104 [==============================] - 0s 2ms/step - loss: 0.3148 - accuracy: 0.7966

In [60]:
# predict and score
predicts = pd.DataFrame(nn.predict(X_test), columns = [0, 1, 2])

model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(classification_report(y_test.idxmax(axis = 1), predicts.idxmax(axis = 1), target_names = target_names))
confusion_matrix(y_test.idxmax(axis = 1), predicts.idxmax(axis = 1))

35/35 [==============================] - 0s 1ms/step
35/35 - 0s - loss: 0.5638 - accuracy: 0.7486 - 127ms/epoch - 4ms/step
Loss: 0.5637906193733215, Accuracy: 0.7486437559127808
              precision    recall  f1-score   support

     Dropout       0.75      0.76      0.76       353
    Graduate       0.84      0.86      0.85       560
    Enrolled       0.44      0.39      0.41       193

    accuracy                           0.75      1106
   macro avg       0.68      0.67      0.67      1106
weighted avg       0.74      0.75      0.74      1106



array([[270,  41,  42],
       [ 25, 483,  52],
       [ 65,  53,  75]], dtype=int64)